# Projeto Machine Learning - Predição de Acidentes Aéreos Fatais

## Sumário do Projeto: Predição de Acidentes Aéreos Fatais

### 1. Introdução e Objetivos
#### 1.1. Contexto do Problema
#### 1.2. Objetivos da Análise
- Análise Exploratória
- Engenharia de Features
- Modelagem Preditiva
- Validação
#### 1.3. Dicionário de Variáveis

### 2. Configuração do Ambiente

### 3. Carga e Análise Inicial dos Dados

### 4. Limpeza e Pré-Processamento dos Dados
#### 4.1. Remoção de Duplicatas
#### 4.2. Tratamento de Tipos de Dados
#### 4.3. Tratamento de Valores Ausentes (Nulos)
- Preenchimento com mediana (numéricos)
- Preenchimento com moda (categóricos)
- Remoção de linhas críticas

### 5. Análise Exploratória de Dados (EDA)
#### 5.1. Distribuição Geográfica dos Acidentes
#### 5.2. Análise Temporal
#### 5.3. Balanceamento das Classes

### 6. Engenharia e Seleção de Features
#### 6.1. Criação de Features Temporais
#### 6.2. Separação de Features Numéricas e Categóricas
#### 6.3. Preparação dos Dados para Modelagem

### 7. Modelagem
#### 7.1. Divisão Treino/Teste
#### 7.2. Encoding e Normalização
#### 7.3. Balanceamento com SMOTE
#### 7.4. Treinamento dos Modelos
- Modelo Baseline (Dummy)
- Regressão Logística
- Árvore de Decisão

### 8. Validação e Comparação dos Modelos
#### 8.1. Métricas de Performance
#### 8.2. Matrizes de Confusão
#### 8.3. Curva ROC e AUC

### 9. Otimização e Avaliação Final
#### 9.1. Análise de Threshold
#### 9.2. Comparação Final dos Modelos
#### 9.3. Importância das Features

### 10. Conclusões e Próximos Passos

---

## Alunos:
- **Eduardo**


##### Link do projeto no GitHub: https://github.com/vtQuadros/Trabalho-Machine-Learning


---

## 1. Introdução e Objetivos

### 1.1 Contexto do Problema

A segurança aérea é uma preocupação fundamental no setor de aviação. Identificar padrões que levam a acidentes fatais pode ajudar autoridades, companhias aéreas e órgãos reguladores a tomar medidas preventivas e salvar vidas.

Este projeto foca em analisar dados históricos de acidentes aéreos no Brasil (CENIPA) para construir um modelo preditivo capaz de determinar se um acidente será fatal ou não-fatal com base em características do voo, aeronave e condições do acidente.

### 1.2 Objetivos da Análise

- **1.** **Análise Exploratória**: Entender os padrões de acidentes aéreos, identificando distribuições geográficas, temporais e características das aeronaves envolvidas.
- **2.** **Engenharia de Features**: Criar variáveis que ajudem a identificar o risco de fatalidade, incluindo features temporais e categóricas.
- **3.** **Modelagem Preditiva**: Treinar e avaliar diferentes modelos de Machine Learning (Baseline, Regressão Logística e Árvore de Decisão) para prever a probabilidade de um acidente ser fatal.
- **4.** **Validação**: Avaliar os modelos usando múltiplas métricas (Acurácia, Precisão, Recall, F1-Score, AUC-ROC) e otimizar o threshold de decisão.

### 1.3 Dicionário de Variáveis

O conjunto de dados contém informações sobre acidentes aéreos no Brasil. As principais variáveis incluem:

**Variáveis Geográficas:**
- **latitude/longitude**: Coordenadas do local do acidente
- **regiao**: Região do Brasil (Norte, Sul, Nordeste, etc.)
- **uf**: Unidade Federativa

**Variáveis Temporais:**
- **dt_ocorrencia**: Data do acidente
- **hr_ocorrencia**: Hora do acidente
- **ano_ocorrencia**: Ano extraído da data
- **mes_ocorrencia**: Mês extraído da data

**Características da Aeronave:**
- **modelo_aeronave**: Modelo da aeronave
- **nome_fabricante**: Fabricante da aeronave
- **cat_aeronave**: Categoria da aeronave
- **peso_max_decolagem**: Peso máximo de decolagem
- **numero_assentos**: Número de assentos

**Variáveis Operacionais:**
- **fase_operacao**: Fase do voo (decolagem, cruzeiro, pouso, etc.)
- **op_padronizado**: Operação padronizada

**Variável Target:**
- **les_fatais_trip**: 1 = Fatal, 0 = Não Fatal

## 2. Configuração do Ambiente

Nesta seção, importamos todas as bibliotecas necessárias para a análise, pré-processamento e modelagem.

In [ ]:
# Bibliotecas para manipulação e análise de dados
import pandas as pd
import numpy as np

# Bibliotecas para visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliotecas de Machine Learning e pré-processamento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay, roc_auc_score, RocCurveDisplay

# Balanceamento de classes
from imblearn.over_sampling import SMOTE

# Configurações de visualização
import warnings
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
%matplotlib inline

print("✓ Bibliotecas importadas com sucesso!")

## 3. Carga e Análise Inicial dos Dados

Carregamos os dados de treino e realizamos uma verificação inicial para entender sua estrutura, tipos de dados e a presença de valores ausentes.

In [ ]:
# Carregamento do dataset de treino
df = pd.read_csv("docs/treino.csv")

print(f"✓ Dados carregados com sucesso!")
print(f"Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")

### Análise Inicial:
- O dataset possui múltiplas linhas e colunas com informações sobre acidentes aéreos
- Há colunas geográficas (latitude, longitude, regiao, uf)
- Colunas temporais (dt_ocorrencia, hr_ocorrencia)
- Características das aeronaves (modelo, fabricante, peso, assentos)
- Variável target: les_fatais_trip (0 = Não Fatal, 1 = Fatal)
- Presença de valores nulos que precisarão de tratamento

In [ ]:
# Amostra dos dados
df.head()

In [ ]:
# Verificando a estrutura e os tipos de dados do DataFrame
df.info()

## 4. Limpeza e Pré-Processamento dos Dados

Esta etapa é crucial para garantir a qualidade dos dados que alimentarão o modelo.

### 4.1 Remoção de Duplicatas

Removemos registros duplicados que podem distorcer a análise.

In [ ]:
# Verificando informações antes da remoção
print("Antes da remoção de duplicatas:")
df.info()

In [ ]:
# Removendo duplicatas
linhas_antes = len(df)
df = df.drop_duplicates().reset_index(drop=True)
linhas_depois = len(df)

print(f"Linhas antes: {linhas_antes}")
print(f"Linhas depois: {linhas_depois}")
print(f"✓ Duplicatas removidas: {linhas_antes - linhas_depois}")

### 4.2 Tratamento de Tipos de Dados

Convertemos colunas para os tipos apropriados (datas, numéricos, etc.).

In [ ]:
# Converter latitude e longitude para float
df['latitude'] = df['latitude'].astype(str).str.replace(',', '.').astype(float)
df['longitude'] = df['longitude'].astype(str).str.replace(',', '.').astype(float)

# Converter data
df['dt_ocorrencia'] = pd.to_datetime(df['dt_ocorrencia'], format='%d/%m/%Y', errors='coerce')

print("Conversões realizadas!")
df.info()

In [ ]:
# Verificar valores nulos
print(df.isnull().sum())

### 4.3 Tratamento de Valores Ausentes (Nulos)

- **Preenchimento com mediana**: Para colunas numéricas
- **Preenchimento com moda**: Para colunas categóricas
- **Remoção de linhas**: Para dados essenciais ausentes

In [ ]:
# Preencher valores ausentes - numéricos com mediana
df['peso_max_decolagem'].fillna(df['peso_max_decolagem'].median(), inplace=True)
df['numero_assentos'].fillna(df['numero_assentos'].median(), inplace=True)

# Preencher valores ausentes - categóricos com moda
df['op_padronizado'].fillna(df['op_padronizado'].mode()[0], inplace=True)
df['hr_ocorrencia'].fillna(df['hr_ocorrencia'].mode()[0], inplace=True)
df['regiao'].fillna(df['regiao'].mode()[0], inplace=True)
df['fase_operacao'].fillna(df['fase_operacao'].mode()[0], inplace=True)
df['modelo_aeronave'].fillna(df['modelo_aeronave'].mode()[0], inplace=True)
df['nome_fabricante'].fillna(df['nome_fabricante'].mode()[0], inplace=True)

# Remover linhas com dados essenciais ausentes
df.dropna(subset=['dt_ocorrencia', 'latitude', 'longitude'], inplace=True)

print("Tratamento de valores ausentes concluído!")
print(f"Total de linhas após tratamento: {len(df)}")
print("\nValores nulos restantes:")
print(df.isnull().sum())

In [ ]:
# Criar novas colunas de ano e mês
df['ano_ocorrencia'] = df['dt_ocorrencia'].dt.year
df['mes_ocorrencia'] = df['dt_ocorrencia'].dt.month

print("Novas colunas criadas!")
df[['dt_ocorrencia', 'ano_ocorrencia', 'mes_ocorrencia']].head()

In [ ]:
# Tratamento dos valores (NaN)

print("\n--- Contagem de valores nulos ANTES do tratamento ---")


# Lista de colunas numéricas para imputar com a mediana
colunas_numericas_nan = ['peso_max_decolagem', 'numero_assentos']
for col in colunas_numericas_nan:
    mediana = df[col].median()
    df[col] = df[col].fillna(mediana)
    print(f"Valores nulos em '{col}' preenchidos com a mediana: {mediana}")

print("-" * 20)

# Lista de colunas categóricas para imputar com a moda
colunas_categoricas_nan = ['op_padronizado', 'hr_ocorrencia', 'regiao', 'fase_operacao', 'modelo_aeronave', 'nome_fabricante']
for col in colunas_categoricas_nan:
    moda = df[col].mode()[0]
    df[col] = df[col].fillna(moda)
    print(f"Valores nulos em '{col}' preenchidos com a moda: '{moda}'")

print("-" * 20)

# Removendo linhas onde dados essenciais ainda são nulos
print("Removendo linhas onde 'dt_ocorrencia', 'latitude' ou 'longitude' são nulos...")
df.dropna(subset=['dt_ocorrencia', 'latitude', 'longitude'], inplace=True)

print("\n--- Contagem de valores nulos DEPOIS do tratamento ---")
print(df.isnull().sum())

## 5. Análise Exploratória de Dados (EDA)

### 5.1 Seleção de Features e Visualização da Distribuição da Variável Target

In [ ]:
# Selecionar features e target
features = ['latitude', 'longitude', 'peso_max_decolagem', 'numero_assentos',
            'fase_operacao', 'cat_aeronave', 'regiao', 'uf', 'modelo_aeronave', 
            'nome_fabricante', 'ano_ocorrencia', 'mes_ocorrencia']

X = df[features]
y = df['les_fatais_trip']

print(f"Features selecionadas: {X.shape[1]}")
print(f"Total de registros: {X.shape[0]}")

# Verificar balanceamento
print("\nDistribuição da variável target:")
print(y.value_counts())

# Visualizar balanceamento
plt.figure(figsize=(8, 5))
y.value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Acidentes Fatais vs Não Fatais')
plt.xlabel('Classe (0=Não Fatal, 1=Fatal)')
plt.ylabel('Quantidade')
plt.xticks(rotation=0)
plt.show()

## 6. Engenharia e Seleção de Features

### 6.1 Divisão dos Dados (Treino/Teste)

In [ ]:
# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y
)

print(f"Conjunto de treino: {len(X_train)} linhas")
print(f"Conjunto de teste: {len(X_test)} linhas")

In [ ]:
# Separar colunas numéricas e categóricas
colunas_numericas = ['latitude', 'longitude', 'peso_max_decolagem', 'numero_assentos', 
                     'ano_ocorrencia', 'mes_ocorrencia']
colunas_categoricas = ['fase_operacao', 'cat_aeronave', 'regiao', 'uf', 
                       'modelo_aeronave', 'nome_fabricante']

print("Colunas numéricas:", colunas_numericas)
print("Colunas categóricas:", colunas_categoricas)

### 6.2 Separação de Features por Tipo

In [ ]:
# Codificar variáveis categóricas com get_dummies
X_train_encoded = pd.get_dummies(X_train, columns=colunas_categoricas)
X_test_encoded = pd.get_dummies(X_test, columns=colunas_categoricas)

# Garantir que treino e teste tenham as mesmas colunas
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

print(f"Features após encoding: {X_train_encoded.shape[1]}")

# Normalizar features numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

print("Pré-processamento concluído!")

### 6.3 Encoding e Normalização

## 7. Modelagem

### 7.1 Balanceamento de Classes com SMOTE

In [ ]:
# Aplicar SMOTE para balancear as classes
print("Antes do SMOTE:")
print(f"Classe 0 (Não Fatal): {sum(y_train == 0)}")
print(f"Classe 1 (Fatal): {sum(y_train == 1)}")

smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

print("\nDepois do SMOTE:")
print(f"Classe 0 (Não Fatal): {sum(y_train_balanced == 0)}")
print(f"Classe 1 (Fatal): {sum(y_train_balanced == 1)}")

# Visualizar balanceamento
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

pd.Series(y_train).value_counts().plot(kind='bar', ax=axes[0], color=['skyblue', 'salmon'])
axes[0].set_title('Antes do SMOTE')
axes[0].set_xlabel('Classe')
axes[0].set_ylabel('Quantidade')

pd.Series(y_train_balanced).value_counts().plot(kind='bar', ax=axes[1], color=['lightgreen', 'lightcoral'])
axes[1].set_title('Depois do SMOTE')
axes[1].set_xlabel('Classe')
axes[1].set_ylabel('Quantidade')

plt.tight_layout()
plt.show()

### 7.2 Treinamento dos Modelos

Treinamos três modelos para comparação:
- **Baseline (Dummy)**: Modelo simples de referência
- **Regressão Logística**: Modelo linear com dados balanceados
- **Árvore de Decisão**: Modelo não-linear

In [ ]:
# Treinar os modelos

# 1. Modelo Baseline (Dummy)
modelo_baseline = DummyClassifier(strategy='most_frequent', random_state=42)
modelo_baseline.fit(X_train_scaled, y_train)

# 2. Regressão Logística com dados balanceados
modelo_logistica = LogisticRegression(random_state=42, max_iter=1000)
modelo_logistica.fit(X_train_balanced, y_train_balanced)

# 3. Árvore de Decisão
modelo_arvore = DecisionTreeClassifier(random_state=42)
modelo_arvore.fit(X_train_scaled, y_train)

print("Modelos treinados com sucesso!")

In [ ]:
# Fazer predições
y_pred_baseline = modelo_baseline.predict(X_test_scaled)
y_pred_logistica = modelo_logistica.predict(X_test_scaled)
y_pred_arvore = modelo_arvore.predict(X_test_scaled)

# Calcular métricas para cada modelo
modelos = ['Baseline', 'Regressão Logística', 'Árvore de Decisão']
predicoes = [y_pred_baseline, y_pred_logistica, y_pred_arvore]

print("=" * 60)
print("RESULTADOS DOS MODELOS")
print("=" * 60)

for nome, y_pred in zip(modelos, predicoes):
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    
    print(f"\n{nome}:")
    print(f"  Acurácia:  {acc:.4f}")
    print(f"  Precisão:  {prec:.4f}")
    print(f"  Recall:    {rec:.4f}")
    print(f"  F1-Score:  {f1:.4f}")

print("=" * 60)

### 7.3 Predições e Métricas Iniciais

## 8. Validação e Comparação dos Modelos

### 8.1 Matrizes de Confusão

In [ ]:
# Matriz de Confusão - Regressão Logística
print("Matriz de Confusão - Regressão Logística")
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_logistica, cmap='Blues')
plt.title('Matriz de Confusão - Regressão Logística')
plt.show()

# Matriz de Confusão - Árvore de Decisão
print("\nMatriz de Confusão - Árvore de Decisão")
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_arvore, cmap='Greens')
plt.title('Matriz de Confusão - Árvore de Decisão')
plt.show()

As matrizes de confusão mostram os acertos e erros de cada modelo:

In [ ]:
# Curva ROC
from sklearn.metrics import RocCurveDisplay

fig, ax = plt.subplots(figsize=(10, 7))

# Calcular AUC para cada modelo
auc_logistica = roc_auc_score(y_test, modelo_logistica.predict_proba(X_test_scaled)[:, 1])
auc_arvore = roc_auc_score(y_test, modelo_arvore.predict_proba(X_test_scaled)[:, 1])

# Plotar curvas
RocCurveDisplay.from_predictions(
    y_test, 
    modelo_logistica.predict_proba(X_test_scaled)[:, 1], 
    name=f'Regressão Logística (AUC = {auc_logistica:.3f})', 
    ax=ax,
    color='blue'
)

RocCurveDisplay.from_predictions(
    y_test, 
    modelo_arvore.predict_proba(X_test_scaled)[:, 1], 
    name=f'Árvore de Decisão (AUC = {auc_arvore:.3f})', 
    ax=ax,
    color='green'
)

ax.plot([0, 1], [0, 1], linestyle='--', color='red', label='Aleatório (AUC = 0.5)')
plt.title('Curva ROC - Comparação dos Modelos')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.show()

print(f"AUC Regressão Logística: {auc_logistica:.3f}")
print(f"AUC Árvore de Decisão: {auc_arvore:.3f}")

### 8.2 Curva ROC e AUC

## 9. Otimização e Avaliação Final

### 9.1 Análise de Threshold (Limiar de Decisão)

Por padrão, o modelo usa threshold de 0.5 para classificar. Vamos testar diferentes thresholds para otimizar o F1-Score.

In [ ]:
# Obtendo as probabilidades de predição
y_proba = modelo_logistica.predict_proba(X_test_scaled)[:, 1]

print("Testando diferentes thresholds de 0.0 a 1.0...")

# Testando diferentes thresholds
thresholds_results = {
    'threshold': [],
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_score': []
}

for threshold in np.arange(0.0, 1.01, 0.01):
    # Aplicando o threshold customizado
    y_pred_threshold = (y_proba >= threshold).astype(int)
    
    # Calculando as métricas
    acc = accuracy_score(y_test, y_pred_threshold)
    prec = precision_score(y_test, y_pred_threshold, zero_division=0)
    rec = recall_score(y_test, y_pred_threshold, zero_division=0)
    f1 = f1_score(y_test, y_pred_threshold, zero_division=0)
    
    # Armazenando os resultados
    thresholds_results['threshold'].append(threshold)
    thresholds_results['accuracy'].append(acc)
    thresholds_results['precision'].append(prec)
    thresholds_results['recall'].append(rec)
    thresholds_results['f1_score'].append(f1)

# Convertendo para DataFrame
df_thresholds = pd.DataFrame(thresholds_results)

# Encontrando o melhor threshold baseado no F1-Score
melhor_threshold_idx = df_thresholds['f1_score'].idxmax()
melhor_threshold = df_thresholds.loc[melhor_threshold_idx, 'threshold']
melhor_f1 = df_thresholds.loc[melhor_threshold_idx, 'f1_score']

print(f"\nMelhor threshold encontrado: {melhor_threshold:.2f}")
print(f"F1-Score: {melhor_f1:.4f}")
print(f"Accuracy: {df_thresholds.loc[melhor_threshold_idx, 'accuracy']:.4f}")
print(f"Precision: {df_thresholds.loc[melhor_threshold_idx, 'precision']:.4f}")
print(f"Recall: {df_thresholds.loc[melhor_threshold_idx, 'recall']:.4f}")

print(f"\nTop 5 melhores thresholds por F1-Score:")
print(df_thresholds.nlargest(5, 'f1_score')[['threshold', 'f1_score', 'accuracy', 'precision', 'recall']])

### 9.2 Visualização do Impacto do Threshold

Vamos visualizar graficamente como o threshold afeta as diferentes métricas.

In [ ]:
# Criando o gráfico de impacto do threshold
plt.figure(figsize=(14, 6))

# Plotando as curvas de métricas
plt.plot(df_thresholds['threshold'], df_thresholds['accuracy'], 
         label='Acurácia', linewidth=2, color='blue', alpha=0.7)
plt.plot(df_thresholds['threshold'], df_thresholds['precision'], 
         label='Precisão', linewidth=2, color='green', alpha=0.7)
plt.plot(df_thresholds['threshold'], df_thresholds['recall'], 
         label='Recall', linewidth=2, color='orange', alpha=0.7)
plt.plot(df_thresholds['threshold'], df_thresholds['f1_score'], 
         label='F1-Score', linewidth=2.5, color='red', alpha=0.9)

# Marcando o melhor threshold
plt.axvline(x=melhor_threshold, color='purple', linestyle='--', linewidth=2, 
            label=f'Melhor Threshold ({melhor_threshold:.2f})')
plt.scatter([melhor_threshold], [melhor_f1], color='purple', s=200, zorder=5, 
            marker='*', edgecolors='black', linewidths=1.5)

# Configurações do gráfico
plt.xlabel('Threshold (Limiar de Decisão)', fontsize=12, fontweight='bold')
plt.ylabel('Score das Métricas', fontsize=12, fontweight='bold')
plt.title('Impacto do Threshold nas Métricas de Avaliação\n(Trade-off entre Precisão e Recall)', 
          fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3, linestyle='--')
plt.xticks(np.arange(0, 1.1, 0.1))
plt.ylim([0, 1.05])

plt.tight_layout()
plt.show()

# Interpretação dos resultados
print("\n📖 INTERPRETAÇÃO DO GRÁFICO:")
print("=" * 70)
print("\n1. PRECISÃO (linha verde):")
print("   - Aumenta conforme o threshold aumenta")
print("   - Threshold alto = menos falsos positivos = maior precisão")
print("   - Use threshold alto quando o custo de falsos positivos é alto")

print("\n2. RECALL (linha laranja):")
print("   - Diminui conforme o threshold aumenta")
print("   - Threshold baixo = menos falsos negativos = maior recall")
print("   - Use threshold baixo quando o custo de falsos negativos é alto")

print("\n3. F1-SCORE (linha vermelha - MAIS IMPORTANTE):")
print("   - Equilibra Precisão e Recall")
print(f"   - Pico em threshold = {melhor_threshold:.2f}")
print("   - É o melhor ponto de equilíbrio entre as duas métricas")

print("\n4. TRADE-OFF:")
print("   - O gráfico mostra claramente o trade-off entre Precisão e Recall")
print("   - Não podemos maximizar ambos simultaneamente")
print(f"   - O threshold ótimo ({melhor_threshold:.2f}) balanceia ambos")

print("\n" + "=" * 70)

### 9.3 Comparação Final dos Modelos

In [ ]:
# Aplicando threshold otimizado
y_pred_logistica_threshold = (y_proba >= melhor_threshold).astype(int)

# Comparando métricas dos modelos
metricas = {
    'Modelo': ['Baseline (Dummy)', 'Regressão Logística', 'Árvore de Decisão'],
    'Acurácia': [
        accuracy_score(y_test, y_pred_baseline), 
        accuracy_score(y_test, y_pred_logistica_threshold),
        accuracy_score(y_test, y_pred_arvore)
    ],
    'Precisão': [
        precision_score(y_test, y_pred_baseline, zero_division=0), 
        precision_score(y_test, y_pred_logistica_threshold, zero_division=0),
        precision_score(y_test, y_pred_arvore, zero_division=0)
    ],
    'Recall': [
        recall_score(y_test, y_pred_baseline, zero_division=0), 
        recall_score(y_test, y_pred_logistica_threshold, zero_division=0),
        recall_score(y_test, y_pred_arvore, zero_division=0)
    ],
    'F1-Score': [
        f1_score(y_test, y_pred_baseline, zero_division=0), 
        f1_score(y_test, y_pred_logistica_threshold, zero_division=0),
        f1_score(y_test, y_pred_arvore, zero_division=0)
    ]
}

df_metricas = pd.DataFrame(metricas)

# Mostrando resultados
print("="*80)
print("COMPARAÇÃO DE MÉTRICAS DOS MODELOS")
print("="*80)
print(df_metricas)

# Encontrando o melhor modelo por F1-Score
melhor_modelo_idx = df_metricas['F1-Score'].idxmax()
print(f"\nMELHOR MODELO: {df_metricas.loc[melhor_modelo_idx, 'Modelo']}")
print(f"F1-Score: {df_metricas.loc[melhor_modelo_idx, 'F1-Score']:.4f}")
print("="*80)

# Matriz de Confusão - Regressão Logística
print("\nMatriz de Confusão: Regressão Logística")
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_logistica_threshold, cmap='Blues')
plt.title(f'Matriz de Confusão - Regressão Logística\n(Threshold = {melhor_threshold:.2f})')
plt.show()

# Matriz de Confusão - Árvore de Decisão
print("\nMatriz de Confusão: Árvore de Decisão")
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_arvore, cmap='Greens')
plt.title('Matriz de Confusão - Árvore de Decisão')
plt.show()

In [ ]:
# Obtendo probabilidades
y_proba_logistica = modelo_logistica.predict_proba(X_test_scaled)[:, 1]
y_proba_arvore = modelo_arvore.predict_proba(X_test_scaled)[:, 1]

# Calculando AUC
auc_logistica = roc_auc_score(y_test, y_proba_logistica)
auc_arvore = roc_auc_score(y_test, y_proba_arvore)

# Plotando as curvas ROC
fig, ax = plt.subplots(figsize=(10, 7))

RocCurveDisplay.from_predictions(
    y_test, 
    y_proba_logistica, 
    name=f'Regressão Logística (AUC = {auc_logistica:.3f})', 
    ax=ax,
    color='blue',
    linewidth=2.5
)

RocCurveDisplay.from_predictions(
    y_test, 
    y_proba_arvore, 
    name=f'Árvore de Decisão (AUC = {auc_arvore:.3f})', 
    ax=ax,
    color='green',
    linewidth=2
)

# Linha de referência
ax.plot([0, 1], [0, 1], linestyle='--', color='red', label='Classificador Aleatório (AUC = 0.5)', linewidth=2)

plt.title('Curva ROC - Comparação dos Modelos', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\nINTERPRETAÇÃO DA CURVA ROC:")
print("- AUC próximo de 1.0: Modelo excelente")
print("- AUC próximo de 0.5: Modelo aleatório")
print(f"- Regressão Logística: {auc_logistica:.3f}")
print(f"- Árvore de Decisão: {auc_arvore:.3f}")

### 9.4 Análise Exploratória - Distribuição Geográfica

In [ ]:
# Visualização Geográfica dos Acidentes
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Gráfico 1: Mapa de dispersão de todos os acidentes
scatter1 = axes[0].scatter(
    df['longitude'], 
    df['latitude'], 
    c=df['les_fatais_trip'],
    cmap='RdYlGn_r',  # Vermelho (fatal) para Verde (não-fatal)
    alpha=0.6,
    s=50,
    edgecolors='black',
    linewidth=0.5
)
axes[0].set_xlabel('Longitude', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Latitude', fontsize=12, fontweight='bold')
axes[0].set_title('🗺️ Distribuição Geográfica dos Acidentes Aéreos\n(Brasil)', 
                  fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Adicionar legenda
cbar1 = plt.colorbar(scatter1, ax=axes[0])
cbar1.set_label('Fatalidade (0=Não Fatal, 1=Fatal)', rotation=270, labelpad=20)

# Gráfico 2: Acidentes por Região
acidentes_por_regiao = df.groupby(['regiao', 'les_fatais_trip']).size().unstack(fill_value=0)
acidentes_por_regiao.plot(kind='bar', ax=axes[1], color=['lightgreen', 'crimson'], width=0.7)
axes[1].set_xlabel('Região', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Número de Acidentes', fontsize=12, fontweight='bold')
axes[1].set_title('📍 Acidentes por Região e Gravidade', fontsize=14, fontweight='bold')
axes[1].legend(['Não Fatal', 'Fatal'], loc='upper right')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45, ha='right')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Estatísticas geográficas
print("\n📊 ESTATÍSTICAS GEOGRÁFICAS")
print("="*70)
print(f"\n🗺️ Total de acidentes: {len(df)}")
print(f"\n📍 Acidentes por região:")
for regiao in df['regiao'].value_counts().index:
    total = len(df[df['regiao'] == regiao])
    fatais = len(df[(df['regiao'] == regiao) & (df['les_fatais_trip'] == 1)])
    taxa = (fatais/total)*100 if total > 0 else 0
    print(f"   {regiao}: {total} acidentes ({fatais} fatais - {taxa:.1f}%)")

In [ ]:
# Análise Temporal dos Acidentes
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Gráfico 1: Acidentes por Ano
acidentes_ano = df.groupby(['ano_ocorrencia', 'les_fatais_trip']).size().unstack(fill_value=0)
acidentes_ano.plot(kind='bar', ax=axes[0, 0], color=['lightgreen', 'crimson'], width=0.8)
axes[0, 0].set_xlabel('Ano', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Número de Acidentes', fontsize=12, fontweight='bold')
axes[0, 0].set_title('📊 Acidentes por Ano', fontsize=14, fontweight='bold')
axes[0, 0].legend(['Não Fatal', 'Fatal'], loc='upper right')
axes[0, 0].set_xticklabels(axes[0, 0].get_xticklabels(), rotation=45, ha='right')
axes[0, 0].grid(axis='y', alpha=0.3)

# Gráfico 2: Tendência de Acidentes Fatais
acidentes_fatais_ano = df[df['les_fatais_trip'] == 1].groupby('ano_ocorrencia').size()
acidentes_totais_ano = df.groupby('ano_ocorrencia').size()
taxa_fatalidade = (acidentes_fatais_ano / acidentes_totais_ano * 100).fillna(0)

axes[0, 1].plot(taxa_fatalidade.index, taxa_fatalidade.values, marker='o', 
                linewidth=3, markersize=8, color='darkred')
axes[0, 1].fill_between(taxa_fatalidade.index, taxa_fatalidade.values, alpha=0.3, color='red')
axes[0, 1].set_xlabel('Ano', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Taxa de Fatalidade (%)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('📈 Tendência da Taxa de Fatalidade ao Longo dos Anos', 
                      fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Gráfico 3: Acidentes por Mês
acidentes_mes = df.groupby(['mes_ocorrencia', 'les_fatais_trip']).size().unstack(fill_value=0)
acidentes_mes.plot(kind='bar', ax=axes[1, 0], color=['lightgreen', 'crimson'], width=0.8)
axes[1, 0].set_xlabel('Mês', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Número de Acidentes', fontsize=12, fontweight='bold')
axes[1, 0].set_title('📆 Acidentes por Mês (Sazonalidade)', fontsize=14, fontweight='bold')
axes[1, 0].legend(['Não Fatal', 'Fatal'], loc='upper right')
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
axes[1, 0].set_xticklabels(meses, rotation=45, ha='right')
axes[1, 0].grid(axis='y', alpha=0.3)

# Gráfico 4: Acidentes por Fase de Operação
acidentes_fase = df.groupby(['fase_operacao', 'les_fatais_trip']).size().unstack(fill_value=0)
acidentes_fase = acidentes_fase.nlargest(10, 1)  # Top 10 fases com mais fatais
acidentes_fase.plot(kind='barh', ax=axes[1, 1], color=['lightgreen', 'crimson'])
axes[1, 1].set_xlabel('Número de Acidentes', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Fase de Operação', fontsize=12, fontweight='bold')
axes[1, 1].set_title('✈️ Top 10 Fases de Operação Mais Críticas', fontsize=14, fontweight='bold')
axes[1, 1].legend(['Não Fatal', 'Fatal'], loc='lower right')
axes[1, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Estatísticas temporais
print("\n📊 ESTATÍSTICAS TEMPORAIS")
print("="*70)
print(f"\n📅 Período analisado: {df['ano_ocorrencia'].min()} - {df['ano_ocorrencia'].max()}")
print(f"\n📈 Taxa média de fatalidade: {taxa_fatalidade.mean():.2f}%")
print(f"\n🔴 Ano com maior taxa de fatalidade: {taxa_fatalidade.idxmax()} ({taxa_fatalidade.max():.2f}%)")
print(f"🟢 Ano com menor taxa de fatalidade: {taxa_fatalidade.idxmin()} ({taxa_fatalidade.min():.2f}%)")
print(f"\n📆 Mês com mais acidentes: {meses[acidentes_mes.sum(axis=1).idxmax()-1]}")
print(f"📆 Mês com menos acidentes: {meses[acidentes_mes.sum(axis=1).idxmin()-1]}")

### 9.5 Análise Exploratória - Padrões Temporais

In [ ]:
# Comparação Visual dos Modelos
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Preparar dados para visualização
modelos_nomes = ['Baseline\n(Dummy)', 'Regressão\nLogística', 'Árvore de\nDecisão']
cores_modelos = ['gray', 'green', 'orange']

# Gráfico 1: Comparação de Acurácia
acuracias = df_metricas['Acurácia'].values
bars1 = axes[0, 0].bar(range(len(modelos_nomes)), acuracias, color=cores_modelos, edgecolor='black', linewidth=1.5)
axes[0, 0].set_ylabel('Acurácia', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Acurácia dos Modelos', fontsize=14, fontweight='bold')
axes[0, 0].set_xticks(range(len(modelos_nomes)))
axes[0, 0].set_xticklabels(modelos_nomes, fontsize=9)
axes[0, 0].set_ylim([0, 1])
axes[0, 0].grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars1):
    height = bar.get_height()
    axes[0, 0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{acuracias[i]:.3f}', ha='center', va='bottom', fontweight='bold')

# Gráfico 2: Comparação de Precisão
precisoes = df_metricas['Precisão'].values
bars2 = axes[0, 1].bar(range(len(modelos_nomes)), precisoes, color=cores_modelos, edgecolor='black', linewidth=1.5)
axes[0, 1].set_ylabel('Precisão', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Precisão dos Modelos', fontsize=14, fontweight='bold')
axes[0, 1].set_xticks(range(len(modelos_nomes)))
axes[0, 1].set_xticklabels(modelos_nomes, fontsize=9)
axes[0, 1].set_ylim([0, 1])
axes[0, 1].grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars2):
    height = bar.get_height()
    axes[0, 1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{precisoes[i]:.3f}', ha='center', va='bottom', fontweight='bold')

# Gráfico 3: Comparação de Recall
recalls = df_metricas['Recall'].values
bars3 = axes[1, 0].bar(range(len(modelos_nomes)), recalls, color=cores_modelos, edgecolor='black', linewidth=1.5)
axes[1, 0].set_ylabel('Recall', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Recall dos Modelos', fontsize=14, fontweight='bold')
axes[1, 0].set_xticks(range(len(modelos_nomes)))
axes[1, 0].set_xticklabels(modelos_nomes, fontsize=9)
axes[1, 0].set_ylim([0, 1])
axes[1, 0].grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars3):
    height = bar.get_height()
    axes[1, 0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{recalls[i]:.3f}', ha='center', va='bottom', fontweight='bold')

# Gráfico 4: Comparação de F1-Score
f1_scores = df_metricas['F1-Score'].values
bars4 = axes[1, 1].bar(range(len(modelos_nomes)), f1_scores, color=cores_modelos, edgecolor='black', linewidth=1.5)
axes[1, 1].set_ylabel('F1-Score', fontsize=12, fontweight='bold')
axes[1, 1].set_title('F1-Score dos Modelos (MÉTRICA PRINCIPAL)', fontsize=14, fontweight='bold')
axes[1, 1].set_xticks(range(len(modelos_nomes)))
axes[1, 1].set_xticklabels(modelos_nomes, fontsize=9)
axes[1, 1].set_ylim([0, 1])
axes[1, 1].grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars4):
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{f1_scores[i]:.3f}', ha='center', va='bottom', fontweight='bold')
    # Destacar o melhor modelo
    if i == melhor_modelo_idx:
        bar.set_edgecolor('gold')
        bar.set_linewidth(4)

plt.tight_layout()
plt.show()

# Resumo estatístico
print("\nRANKING DOS MODELOS (por F1-Score)")
print("="*70)
ranking = df_metricas.sort_values('F1-Score', ascending=False)
for idx, row in ranking.iterrows():
    emoji = "1º" if idx == 0 else "2º" if idx == 1 else "3º"
    print(f"{emoji} {row['Modelo']}")
    print(f"   F1-Score: {row['F1-Score']:.4f} | Acurácia: {row['Acurácia']:.4f} | "
          f"Precisão: {row['Precisão']:.4f} | Recall: {row['Recall']:.4f}\n")

### 9.6 Comparação Visual dos Modelos

In [ ]:
# Análise de Importância das Features
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Gráfico 1: Importância das Features - Regressão Logística
# Usar coeficientes do modelo de Regressão Logística
feature_names_encoded = X_train_encoded.columns.tolist()
coeficientes = modelo_logistica.coef_[0]

# Criar DataFrame com importâncias
importancias_log_df = pd.DataFrame({
    'Feature': feature_names_encoded,
    'Importância': np.abs(coeficientes)  # Valor absoluto para ranking
}).sort_values('Importância', ascending=False).head(15)

# Plotar
axes[0].barh(range(len(importancias_log_df)), importancias_log_df['Importância'], 
             color='green', edgecolor='black')
axes[0].set_yticks(range(len(importancias_log_df)))
axes[0].set_yticklabels(importancias_log_df['Feature'], fontsize=9)
axes[0].set_xlabel('Importância Absoluta (|Coeficiente|)', fontsize=12, fontweight='bold')
axes[0].set_title('Top 15 Features - Regressão Logística', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(axis='x', alpha=0.3)

# Gráfico 2: Importância das Features - Árvore de Decisão
importancias_arvore = modelo_arvore.feature_importances_

importancias_arvore_df = pd.DataFrame({
    'Feature': feature_names_encoded,
    'Importância': importancias_arvore
}).sort_values('Importância', ascending=False).head(15)

axes[1].barh(range(len(importancias_arvore_df)), importancias_arvore_df['Importância'], 
             color='orange', edgecolor='black')
axes[1].set_yticks(range(len(importancias_arvore_df)))
axes[1].set_yticklabels(importancias_arvore_df['Feature'], fontsize=9)
axes[1].set_xlabel('Importância (Gini)', fontsize=12, fontweight='bold')
axes[1].set_title('Top 15 Features - Árvore de Decisão', fontsize=14, fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Resumo das features mais importantes
print("\nFEATURES MAIS IMPORTANTES")
print("="*70)
print("\nRegressão Logística (Top 5):")
for idx, row in importancias_log_df.head(5).iterrows():
    print(f"   {row['Feature']}: {row['Importância']:.4f}")

print("\nÁrvore de Decisão (Top 5):")
for idx, row in importancias_arvore_df.head(5).iterrows():
    print(f"   {row['Feature']}: {row['Importância']:.4f}")

### 9.7 Importância das Features